# Week 2 Exercise

## Objective:

Demonstrate what has been learnt in week 2 by upgrading week 1 project to have a UI using Gradio UI. Expected to include streaming and use of system prompts to add expertise and ability to switch between models. 
Bonus points if use of a tool can also be demonstrated.
Audio input with autio output also a bonus.

In [ ]:
# imports

import os
from dotenv import load_dotenv
import gradio as gr
import anthropic
import google.generativeai as genai
from openai import OpenAI

In [ ]:
load_dotenv(override=True)

# Set up the Anthropic API key
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
if anthropic_api_key:
    print(f"Anthropic API key set and begins with: {anthropic_api_key[:6]}...")

# Set up the Google API key
google_api_key = os.getenv("GOOGLE_API_KEY")
if google_api_key:
    print(f"Google API key set and begins with: {google_api_key[:6]}...")

openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)

In [ ]:
# models

MODEL_LLAMA="llama3.2"
MODEL_ANTHROPIC="claude-sonnet-4-5-20250929"
MODEL_GOOGLE="gemini-2.5-flash"


In [ ]:
system_message = """
You are an expert software engineer.
You are given a technical question and you need to explain what the code does and why.
"""

In [ ]:
MODEL=MODEL_LLAMA

In [ ]:
def stream_llama(message):
  history = []
  history = [{"role":h["role"], "content":h["content"]} for h in history]
  
  messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

  stream = openai.chat.completions.create(model=MODEL_LLAMA, messages=messages, stream=True)

  response = ""
  for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    yield response

def stream_claude(message):
  history = []
  history = [{"role":h["role"], "content":h["content"]} for h in history]
  
  messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

  stream = anthropic.chat.completions.create(model=MODEL_ANTHROPIC, messages=messages, stream=True)

  response = ""
  for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    yield response
  
def stream_gemini(message):
  history = []
  history = [{"role":h["role"], "content":h["content"]} for h in history]
  
  messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

  stream = gemini.chat.completions.create(model=MODEL_GOOGLE, messages=messages, stream=True)

  response = ""
  for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    yield response


In [ ]:
def stream_model(prompt, model):
  print(f"Prompt: {prompt}, Model: {model}")

  if model=="Llama":
    result = stream_llama(prompt)
  elif model=="Claude":
    result = stream_claude(prompt)
  elif model=="Gemini":
    result = stream_gemini(prompt)
  else:
    raise ValueError("Unknown model")
  yield from result

In [ ]:
question_input = gr.Textbox(label="Your message:", info="Enter a question", lines=7)
model_selector = gr.Dropdown(choices=["Llama", "Claude", "Gemini"], value="Llama", label="Model") 
message_output = gr.Markdown(label="Response:")

In [ ]:
view = gr.Interface(
  fn=stream_model,  
  inputs=[question_input, model_selector], 
  outputs=message_output,
  flagging_mode="never"
  )

view.launch(inbrowser=True)